## Price an arithmetic average call using the binomial tree model.
###   Payoff(t) = max(Save(t) - K; 0) ；  Save(t) = arithmetic average of stock prices
### 1. Binomial tree model
### 2. Monte Carlo simulation


##  Binomial tree model

In [9]:
# Price an arithmetic average call using the binomial tree model.
import numpy as np
import math

# Input Info
s_p = 50
strike = 50
q = 0.0
sd1 = 0.4
r = 0.1
t = 0.25
t1 = 1
period = 100
period_pass = (t / (t1 - t)) * period
savet = 70
M = 5

delt =  (t1 - t) / period
u =  np.exp(sd1 * (delt**0.5))   
d =  1/u         
# pre-calculate
p =   (np.exp((r-q)*delt) - d)/(u - d)
print(u,d,p)

s_price = [] # list of stock prices tree
s_ave = [] # list of max stock prices tree 
o_price_e = [] # list of option prices tree for european option
o_price_a = [] # list of option prices tree for american option
aul = []
adl = []
probability_rate = []
for i in range(period+1):
    s_price.append([])
    o_price_e.append([])
    o_price_a.append([])
    s_ave.append([])
    aul.append([])
    adl.append([])

for i in range(period+1):
    for j in range(i+1):
        o_price_e[i].append([])
        o_price_a[i].append([])
        s_ave[i].append([])
        aul[i].append([])
        adl[i].append([])



# stock prices and probabilities for all periods
for j in range(period+1):
    for g in range(j+1):
        s_price[j].append(round(s_p*(u**(j-g))*((d)**(g)),10)) # Calculate Stock Price
        
for i in range(int(period/2)+1):
    s_price[i*2][i] = s_p

for i in range(len(s_price)):
    for j in range(len(s_price[i])):
        smax_temp = (savet*(period_pass+1)+s_p * u*(1-u**(i-j))/(1-u)+s_p*(u**(i-j))*d*(1-d**j)/(1-d))/(i+period_pass+1)
        smin_temp = (savet*(period_pass+1)+s_p * d*(1-d**j)/(1-d)+s_p*(d**(j))*u*(1-u**(i-j))/(1-u))/(i+period_pass+1)
        for k in range(M+1):
            s_ave[i][j].append(round((((M-k)/M)*smax_temp+(k/M)*smin_temp),11))

# print(smax_temp,smin_temp)
# terminal payoff
for j in range(len(s_ave[-1])):
    for k in range(len(s_ave[-1][j])):
        o_price_e[-1][j].append(max(s_ave[-1][j][k]-strike,0))
        o_price_a[-1][j].append(max(s_ave[-1][j][k]-strike,0))

# Generate list of Au and Ad
for i in range(len(s_price)):
    for j in range(len(s_price[i])):
        for k in range(M+1):
            aul[i][j].append(round(((i+period_pass+1)*s_ave[i][j][k]+s_p*u**(i+1-j)*d**j)/(i+period_pass+2),5))
            adl[i][j].append(round(((i+period_pass+1)*s_ave[i][j][k]+s_p*u**(i-j)*d**(j+1))/(i+period_pass+2),5))
# adjust sav
for i in range(len(s_ave)):
    for j in range(len(s_ave[i])):
        for k in range(len(s_ave[i][j])):
            s_ave[i][j][k] = round(s_ave[i][j][k],5)
# print(len(s_ave[-2][-1]))
# print(len(aul[-2][-1]))


# Interpolation method
for i in range(period):
    for j in range(len(s_ave[period - i])-1):
        for k in range(M+1):
            for h in range(M):
                if s_ave[period - i][j][h] >= aul[period-i-1][j][k] and aul[period-i-1][j][k] >= s_ave[period- i][j][h+1]:
                    if s_ave[period - i][j][h] ==  s_ave[period- i][j][h+1]:
                        wu = 1
                    else:
                        wu = (s_ave[period - i][j][h] - aul[period -i-1][j][k]) / (s_ave[period - i][j][h] - s_ave[period- i][j][h+1])
                    break

            for h1 in range(M):
                if s_ave[period - i][j+1][h1] >= adl[period - i-1][j][k] and adl[period - i-1][j][k] >= s_ave[period - i][j+1][h1+1]:
                    if s_ave[period - i][j+1][h1] == s_ave[period - i][j+1][h1+1]:
                        wd = 1
                    else:
                        wd = (s_ave[period - i][j+1][h1] - adl[period - i-1][j][k]) / (s_ave[period - i][j+1][h1] - s_ave[period - i][j+1][h1+1])
                    break
            if aul[period-i-1][j][k] > s_ave[period - i][j][0] or s_ave[period- i][j][-1] > aul[period-i-1][j][k]:
                print("check 111",i,j,k, aul[period-i-1][j][k],s_ave[period - i][j][0],s_ave[period- i][j][-1],aul[period-i-1][j][k])
            if adl[period - i-1][j][k] >  s_ave[period - i][j+1][0] or s_ave[period - i][j+1][-1] > adl[period - i-1][j][k]:
                print("check 222", adl[period - i-1][j][k],s_ave[period - i][j+1][0],s_ave[period - i][j+1][-1])
            c = (p*(wu*o_price_e[period - i][j][h+1]+(1-wu)*o_price_e[period - i][j][h])+(1-p)*(wd*o_price_e[period - i][j+1][h1+1]+(1-wd)*o_price_e[period - i][j+1][h1]))*np.exp(-r*delt)
            c2 = max(s_ave[period - i-1][j][k] - strike,(p*(wu*o_price_a[period - i][j][h+1]+(1-wu)*o_price_a[period - i][j][h])+(1-p)*(wd*o_price_a[period - i][j+1][h1+1]+(1-wd)*o_price_a[period - i][j+1][h1]))*np.exp(-r*delt))
            #print(wu,wd,c)
            o_price_e[period - i-1][j].append(c)
            o_price_a[period - i-1][j].append(c2)


# print(o_price_e)   
print("Arithmetic average call with the binomial tree model.")
print("binomial euro call",o_price_e[0][0][0])
print("binomial ame call",o_price_a[0][0][0])


1.035248004772712 0.9659521152320881 0.5021678243676723
Arithmetic average call with the binomial tree model.
binomial euro call 9.666326058778207
binomial ame call 20.0


## Monte Carlo simulation

In [8]:
# Calculate Price of arithmetic average call by Monte Carlo
from scipy.stats import norm
import math
import numpy as np
size = 10000
# Input Info

p1 = 50
r1 = 0.1
q1 = 0.0
sd1 = 0.4
strike = 50
t = 0.5
t1 = 1.5
savet = 50
period = 100

period_pass = (t / (t1 - t)) * period
delt = (t1 - t) / period
# call 
vari = (sd1**2)*delt
return_l = []
ans_l = []
s_l = []
pre_ave_l = []
pre_calcu = (r1-q1-((sd1**2)/2))*delt
if t != 0:
    for i in range(int(period_pass)):
        pre_ave_l.append(savet)
pre_ave_l.append(p1)
print(pre_ave_l)
for i in range(20):
    return_l = []
    for j in range(size):
        s_l = []
        for d in range(len(pre_ave_l)):
            s_l.append(pre_ave_l[d])
        ran = np.random.standard_normal(period)
        for g in range(period):
            mean = math.log(s_l[-1])+pre_calcu
            s_l.append(np.exp(mean+float(ran[g])*(vari**0.5)))    # 求出各路徑node

        return_l.append(max(np.mean(s_l)-strike,0))

    ans_l.append(np.mean(return_l)*np.exp(-r1*(t1 - t)))
ans_mean = np.mean(ans_l)
ans_std = np.std(ans_l)
# print("1",return_l)
# print("2",ans_l)
#print(s_l)
print("Calculate Price of European Option by Monte Carlo")
print("put price : ",round(ans_mean,4))
print("95% interval of call price : ",ans_mean- 2*ans_std,ans_mean+2*ans_std)

Calculate Price of European Option by Monte Carlo
put price :  3.7171
95% interval of call price :  3.633524650504759 3.8007208303286966
